In [1]:
import cv2
import streamlit as st
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import numpy as np
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
import pandas as pd
import time
import os
import statistics
import webbrowser
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [8]:
# Load CSV file containing links for different emotions
df = pd.read_csv('data/kaggle_song-features.csv')


In [9]:
# Step 3: Assign emotion labels based on valence and energy
def assign_emotion(row):
    if row['valence'] >= 0.505 and row['energy'] >= 0.505:
        return 'happy'
    elif row['valence'] > 0.495 and row['energy'] < 0.495:
        return 'neutral'
    elif row['valence'] < 0.495 and row['energy'] > 0.495:
        return 'angry'
    else:
        return 'sad'
    
df['emotion'] = df.apply(assign_emotion, axis=1)

In [ ]:
def assign_emotion(danceability, valence, energy, tempo):
    if danceability > 0.6 and valence > 0.6 and energy > 0.6:
        return 'happy'
    elif danceability < 0.4 and valence < 0.4 and energy < 0.4:
        return 'sad'
    elif energy > 0.7 and tempo > 120:
        return 'angry'
    else:
        return 'neutral'

# Add a new column 'emotion' to the dataset
df['emotion'] = df.apply(lambda row: assign_emotion(row['danceability'], row['valence'], row['energy'], row['tempo']), axis=1)

# Save the updated dataset to a new CSV file
df.to_csv('songs_with_emotions.csv', index=False)

In [11]:
emotion = 'happy'

In [12]:
df = df[df['emotion'] == emotion]

df.head()

,Unnamed: 0,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,...,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,emotion
27,27,0.723,1921,0.388,['Mehmet Kemiksiz'],0.685,155063,0.698,0,0osXBirvQzPRfKSUDzHPCv,...,4,0.0768,-8.184,0,Korkma Sönmez,0,1921,0.0421,133.951,happy
42,42,0.525,1921,0.258,['Mehmet Kemiksiz'],0.689,145371,0.537,0,1WOXWCjKZzJJRSMJcrLrK1,...,1,0.0468,-9.843,0,Ey Bülbül-i Ter Zebân-i İrf,0,1921,0.0282,107.840,happy
46,46,0.752,1921,0.709,['Mehmet Kemiksiz'],0.516,249652,0.576,0,1qmXkw1WhSvvIL5IqBnW82,...,6,0.1100,-7.197,0,Kasr-ı Gülşen,0,1921,0.0303,123.853,happy
58,58,0.538,1921,0.757,['Mehmet Kemiksiz'],0.550,158642,0.515,0,2MtcWKwc16dqFqdM7v1WST,...,11,0.3750,-6.064,0,Şu Boğaz Harbi Nedir,0,1921,0.0336,123.983,happy
72,72,0.817,1921,0.990,['Darüttalim-i Musiki Heyeti'],0.380,192601,0.525,0,2t5UabjiPwVUXJtcpRHreq,...,1,0.4380,-12.112,0,Şehnaz Longa,0,1921-02-20,0.0432,136.156,happy


In [13]:

seed_song = df.sample()
seed_song

,Unnamed: 0,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,...,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,emotion
99992,99992,0.599,1976,0.344,['Harry Chapin'],0.51,439813,0.696,0,6e9SGey9OqCNFPNbPQU3WK,...,2,0.973,-11.525,1,Circle - Live 1975 Version,29,1976,0.083,118.996,happy


In [14]:
seed_tracks = seed_song['id'].iloc[0]
seed_tracks


'6e9SGey9OqCNFPNbPQU3WK'

In [15]:
def save_token():
    load_dotenv('../.env')
    CLIENT_ID = os.environ.get('CLIENT_ID')
    CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
    command = f"""echo "CLIENT_ID='{CLIENT_ID}'\nCLIENT_SECRET='{CLIENT_SECRET}'" > .env"""
    os.system(command)

In [16]:
def spotify_token ():
    
    """This function refreshes a 'queries token' for a given app on Spotify.
    queries token are refreshed every 3600 seconds
    returns: token as a string
    """

    # 1. Defining: credentials fot the app
    
    CLIENT_ID = os.getenv("CLIENT_ID")
    CLIENT_SECRET = os.getenv("CLIENT_SECRET")
    
    #2. Request
    
    body_params = {"grant_type":"client_credentials"}
    url = "https://accounts.spotify.com/api/token"
    response = requests.post(url, data=body_params, auth=(CLIENT_ID,CLIENT_SECRET))
    
    try:
        token = response.json()["access_token"]
        return token

    except:
        print("The request did not go through. Maybe check credentials?")

In [17]:
token = spotify_token()
token

'BQCA8aLwH_JzFxkroJ7ujr9oAPeMGvHRX_miBibKlRpLmDtJNL2wVnaiQXaDURIsD7wLInrdn2TGRaE1ICGmigGaI9mU4wZVZLEe6eYXMq1uFYpwPos2'

In [19]:
CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Define the seed artist and genre
seed_tracks = []
seed_tracks.append(seed_song['id'].iloc[0])
print(seed_tracks)
#Pop,Hip-Hop,Rock,Indie,R&B,Electronic,Dance,Latin,Alternative,Country,Reggaeton
seed_genres = ['rock', 'pop', 'hip-hop','electronic']

# Define the target energy and valence values
target_energy = float(seed_song['valence'].iloc[0])
target_valence = float(seed_song['energy'].iloc[0])

# Call the recommendation API
results = sp.recommendations(seed_tracks=seed_tracks, seed_genres=seed_genres, target_energy=target_energy, target_valence=target_valence)

# Extract the recommended tracks from the response
recommended_tracks = results['tracks']
#print(recommended_tracks)
#print(results['tracks'][0]['album']['artists'][0]['external_urls']['spotify'])

# Print the recommended tracks
for track in recommended_tracks:
    if 'album' in track and 'artists' in track['album'] and 'external_urls' in track and 'spotify' in track['external_urls']:
        track_info = {
            'name': track['name'],
            'artist': track['album']['artists'][0]['name'],
            'album': track['album']['name'],
            'track_link': track['external_urls']['spotify'],
            'artists': track['album']['artists'],
            'available_markets': track['available_markets'],
            'disc_number': track['disc_number'],
            'duration_ms': track['duration_ms'],
            'explicit': track['explicit'],
            'external_ids': track['external_ids'],
            'href': track['href'],
            'id': track['id'],
            'is_local': track['is_local'],
            'popularity': track['popularity'],
            'preview_url': track['preview_url'],
            'track_number': track['track_number'],
            'type': track['type'],
            'uri': track['uri']
        }
        recommended_tracks.append(track_info)
recommended_tracks_df = pd.DataFrame(recommended_tracks)
recommended_tracks_df.dropna(subset=['track_link'],inplace=True)
recommended_tracks_df.track_link.sample()
link = recommended_tracks_df.track_link.sample().values[0]
link
webbrowser.open_new_tab(link)

['6e9SGey9OqCNFPNbPQU3WK']


True